import packages
------------------

Some packages are installed automatically if you use Anaconda. As pytorch is used here, you are expected to install that in your machine. 

In [1]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# added to be able to run in Google Colab
import sys
sys.path.append('/content/gdrive/MyDrive/UTRECHT/utils')
sys.path.insert(0,'/content/gdrive/MyDrive/UTRECHT')

import utils

In [3]:
from __future__ import print_function, division
import os, random, time, copy
from skimage import io, transform, morphology, feature, measure
from skimage.morphology import dilation, square, binary_opening
from skimage.draw import rectangle_perimeter, circle_perimeter, disk
import numpy as np
import os.path as path
import scipy.io as sio
import scipy
from scipy import misc
from scipy import ndimage, signal
from scipy.ndimage import gaussian_filter
import pickle
import sys
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL.Image
from io import BytesIO
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import pandas as pd 
import matplotlib.patheffects as path_effects
import json

from skimage.filters import threshold_otsu

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

# sys.path.append('/home/skong2/project/dpff4ldl')
# from utils.metrics import *
# from losses import *

from utils.flow_functions import *
from utils.dataset import *
from utils.network_arch import *
from utils.trainval_detSegDistTransform import *

import warnings # ignore warnings
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)

3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
1.13.1+cu116




Evaluation and Visualization
-----

In [4]:
#### set project name, save directory for the project

exp_dir = '/content/gdrive/MyDrive/exp/05_Exp3_ContinuousTraining' 

trial_list = ['Trial_01', 'Trial_02','Trial_03','Trial_04','Trial_05']
time_step_list =['T0','T1','T2','Full']
trial = trial_list[1]    # set to the trial number
time_step = time_step_list[2] # set to time step number
t_minus_1_time_step = time_step_list[1] # set to previous time step

save_dir = os.path.join(exp_dir, trial, time_step) 

eval_dir = '/content/gdrive/MyDrive/eval/05_Exp3_ContinuousTraining' 
eval_dir = os.path.join(eval_dir, trial, time_step) 
if not os.path.exists(eval_dir): 
    os.makedirs(eval_dir)

print(time_step)
print(t_minus_1_time_step)
print(eval_dir)

T2
T1
/content/gdrive/MyDrive/eval/05_Exp3_ContinuousTraining/Trial_02/T2


In [5]:
# cpu or cuda
device ='cpu'
if torch.cuda.is_available(): 
    device='cuda:0'
print(device)

freqShow = 50
weight_reg = 2.0    # balances regression loss with segmentation loss. Value chosen based on past investigation.
weight_background = 0.1   # for regression loss only, downweights background pixels to highlight foreground pollen

#model parameters
batch_size = 4
newSize = [800,800] # set to crop size, to tell model what size tensor to expect
total_epoch_num = 100  # total number of epoch in training
base_lr =0.0005        #0.0005      # base learning rate/
scaleList = [0]      # the number of output layer for U-net
#scale = [0,1,2,3]      # the number of output layer for U-net

cuda:0


In [6]:
# path_to_image = '/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999/C6/C6_tiles_withAnnot'
path_to_image = '/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999'
with open(os.path.join(save_dir,'dbinfo.pkl'), 'rb') as handle:
    dbinfo = pickle.load(handle)    
    
dbinfo.keys(), len(dbinfo['train_det_list']), len(dbinfo['test_det_list'])

(dict_keys(['train_det_list', 'test_det_list']), 240, 60)

In [7]:
################## init model ###################
path_to_save_paramOnly = path.join(exp_dir, trial, t_minus_1_time_step, 'bestValModel_encoder.paramOnly')

curmodel = PollenDet_SegDistTransform(34, scaleList=scaleList, pretrained=False)
curmodel.encoder.encoder.conv1 = nn.Conv2d(27, 64, (7, 7), (2, 2), (3, 3), bias=False) #change dimensions of the first layer in the encoder
curmodel.load_state_dict(torch.load(path_to_save_paramOnly)) #, map_location=torch.device('cpu')
curmodel.to(device);    
#print(curmodel.state_dict)
curmodel.eval()
#curmodel.train()
curmodel.training = False
print(curmodel.training)
print(path_to_save_paramOnly)

False
/content/gdrive/MyDrive/exp/05_Exp3_ContinuousTraining/Trial_02/T1/bestValModel_encoder.paramOnly


In [8]:
class PollenDet4Eval(Dataset):
    def __init__(self, path_to_image='/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999',
                #  path_to_annot='/content/gdrive/MyDrive/UTRECHT/Detection/AnnotCombo9_noNPP_circleMask',
                 path_to_mask='/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999',
                 dbinfo=dbinfo,
                 size=[1040,1392], 
                 set_name='test'):
        
        self.path_to_image = path_to_image
        # self.path_to_annot = path_to_annot
        self.path_to_mask = path_to_mask
        self.transform = transform
        self.dbinfo = dbinfo
        if set_name=='val':
            set_name = 'test'
        self.set_name = set_name        
        self.size = size
        self.resizeFactor = size[0]/1000
        
        self.sampleList = self.dbinfo[set_name+'_det_list']

        self.TFNormalize = transforms.Normalize([0.5] * 27, [0.5]*27)
        self.current_set_len = len(self.sampleList)
        
        self.TF2tensor = transforms.ToTensor()
        self.TF2PIL = transforms.ToPILImage()
        self.TFresize = transforms.Resize((self.size[0],self.size[1]))

    def __len__(self):        
        return self.current_set_len
    
    def __getitem__(self, idx):        
        current_example= self.sampleList[idx] 

        current_image_path= os.path.join(self.path_to_image, current_example[0], current_example[0] + '_tiles_withAnnot', current_example[1],current_example[2])
        current_distTransform_path=os.path.join(self.path_to_image, current_example[0], current_example[0] + '_masks2', current_example[1],current_example[2])
        # curPickleName = path.join(self.path_to_annot, current_example)

        imagestack_array = []
        for file in sorted(os.listdir(current_image_path)):
          if file.endswith('.png'):
            slice = Image.open(os.path.join(current_image_path, file))
            imagestack_array.append(np.asarray(slice))
        image = np.block(imagestack_array)
        if image.shape[2] <27:
          pad_val = 27-image.shape[2]
          npad = ((0, 0), (0,0), (0,pad_val))
          image= np.pad(image, pad_width=npad, mode='constant', constant_values=0)

        for file in sorted(os.listdir(current_distTransform_path)):
          if file.endswith('.png'):
            mask = Image.open(os.path.join(current_distTransform_path, file))
            mask=np.expand_dims(mask, axis=2)           

        label = np.copy(mask)         # 11/30/21 added
        label[label > 0] = 1          # 11/30/21 added       

        image_label = np.concatenate((image, label, mask), axis=2) #12/13/21 added


        mask_distanceTransform = np.copy(mask)     #11/18/21 edited
        
        labelOrgSize = np.copy(mask)
        labelOrgSize = torch.from_numpy(labelOrgSize).unsqueeze(0).unsqueeze(0).squeeze(4)

        mask_distanceTransform = mask_distanceTransform.astype(np.float32)/100.0/self.resizeFactor  # factor=size[0]/1000

        image = self.TF2tensor(image)
        label = torch.from_numpy(label).unsqueeze(0) # self.TF2tensor(label)       
        mask_distanceTransform = torch.from_numpy(mask_distanceTransform).unsqueeze(0) # self.TF2tensor(mask_distanceTransform)

        image = image.unsqueeze(0)
        label = label.unsqueeze(0)        
        mask_distanceTransform = mask_distanceTransform.unsqueeze(0)       

        height,width,layer = image_label.shape
        crop0 = image_label[0:800, 0:800,:]
        crop1 = image_label[0:800, width-800:,:]
        crop2 = image_label[height-800:, 0:800,:]
        crop3 = image_label[height-800:, width-800:,:]

        croplist = [crop0, crop1, crop2, crop3]
        imgList=[]
        labelList=[]
        mask_DTList=[]

        for idx2 in range(len(croplist)):
          image_label = croplist[idx2]
          image_label = self.TF2tensor(image_label)
          image_label = image_label.unsqueeze(0)  
          image = torch.narrow(image_label, 1, 0, image_label.shape[1]-2) 
          label=torch.narrow(image_label, 1, image_label.shape[1]-2, 1) 
          mask_distanceTransform=torch.narrow(image_label, 1, image_label.shape[1]-1, 1)

          image_label = image_label.squeeze(0)
          image = image.type(torch.float)
          image = self.TFNormalize(image)

          imgList.append(image)
          labelList.append(label)
          mask_DTList.append(mask_distanceTransform)

        image = torch.concat(imgList)
        label = torch.concat(labelList)
        mask_distanceTransform = torch.concat(mask_DTList)

        image = image.squeeze(0)
        label = label.squeeze(0)
        mask_distanceTransform = mask_distanceTransform.squeeze(0)
        labelOrgSize = labelOrgSize.squeeze(0)

        return image, label, mask_distanceTransform, labelOrgSize, current_example
        # return image, label, mask_distanceTransform, mask_overlap, mask_voteX, mask_voteY, mask_peaks, mask_radius, labelOrgSize, mask_peaksOrgSize, mask_distanceTransformOrgSize, mask_radiusOrgSize

In [9]:
# set_name = 'test'
set_name = 'train'
det_datasets = PollenDet4Eval(path_to_image=path_to_image,
                              # path_to_annot=path_to_annotCombo,
                              dbinfo=dbinfo, size=newSize, set_name=set_name)

dataloaders = DataLoader(det_datasets,
                         batch_size=1,
                         shuffle=True, 
                         num_workers=8) # num_work can be set to batch_size



In [10]:
def create_circular_mask(mask, center, radius, value=1):
    h, w = mask.shape[:2]
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((Y - center[0])**2 + (X-center[1])**2)

    tmpMask = dist_from_center <= radius
    mask[tmpMask] = value
    
    return mask

In [11]:
def IOU(box1, box2):
    """
We assume that the box follows the format:
box1 = [x1,y1,x2,y2], and box2 = [x3,y3,x4,y4],
where (x1,y1) and (x3,y3) represent the top left coordinate,
and (x2,y2) and (x4,y4) represent the bottom right coordinate
    """
    x1, y1, x2, y2 = box1	
    x3, y3, x4, y4 = box2
    
    assert x1 < x2
    assert y1 < y2
    assert x3 < x4
    assert y3 < y4

    # determine the coordinates of the intersection rectangle
    x_left = max(x1, x3)
    y_top = max(y1, y3)
    x_right = min(x2, x4)
    y_bottom = min(y2, y4)

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left + 1) * (y_bottom - y_top + 1)
    
    # compute the area of both AABBs
    bb1_area = (x2 - x1) * (y2 - y1)
    bb2_area = (x4 - x3) * (y4 - y3)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [12]:
def nms(boxes, conf_threshold=0.1, iou_threshold=0.5):
    bbox_list_thresholded = []
    bbox_list_new = []
    bbox_list_new_txt = []
    # detMask_info = detMask_info

    # Stage 1: sort boxes, filter out boxes with low confidence
    boxes_sorted = sorted(boxes, reverse=True, key = lambda x : x[1])
    for box in boxes_sorted:
        if box[1] > conf_threshold:
            bbox_list_thresholded.append(box)
        else:
            pass
    # Stage 2: loop through the boxes, remove boxes with high IoU
    while len(bbox_list_thresholded) > 0:
        current_box = bbox_list_thresholded.pop(0)
        bbox_list_new.append(current_box)
        current_box_txt = (current_box[0], str(current_box[1]) , str(current_box[2]), str(current_box[3]), str(current_box[4]), str(current_box[5]) )
        current_box_txt = ' '.join(current_box_txt)
        bbox_list_new_txt.append(current_box_txt)

        for box in bbox_list_thresholded:
            if current_box[0] == box[0]:
                iou = IOU(current_box[2:], box[2:])
                # print(iou)
                if iou > iou_threshold:
                    bbox_list_thresholded.remove(box)
                    # detMask_info.remove()

    return  bbox_list_new, bbox_list_new_txt

# Create corrected prediction masks

In [13]:
# project_name = '05_Exp3_ContinuousTraining'
# model = [t_minus_1_time_step + '_model',time_step + '_model']

In [21]:
# pred_masks_corrected = os.path.join('/content/gdrive/MyDrive/UTRECHT/Detection/PAL1999/',current_example[0][0], 'pred_masks_corrected', current_example[1][0])
pred_mask_dir = os.path.join(save_dir, 'pred_masks') 

if not os.path.exists(pred_mask_dir): 
    os.makedirs(pred_mask_dir)
print(pred_mask_dir)

/content/gdrive/MyDrive/exp/05_Exp3_ContinuousTraining/Trial_02/T2/pred_masks


In [22]:
iterCount, sampleCount = 0, 0
phase = 'train' # 'train'
for sample in dataloaders: 
            
    curImg, curLabel, curMask, curMask_orgSize, current_example = sample
    
    curImg, curLabel, curMask, curMask_orgSize = curImg.to(device), curLabel.to(device), curMask.to(device), curMask_orgSize.to(device)

    curImg_squeeze=torch.squeeze(curImg,0)

    iterCount += 1
    sampleCount += curLabel.size(0)   
    
    outputs = curmodel(curImg_squeeze)
    predSeg = outputs[('segMask', 0)]
    predDistTransform = outputs[('output', 0)]

    softmax = predSeg
    
    if iterCount%50==0:
        print('{}/{}'.format(iterCount,len(det_datasets)))

    mask_dir =  os.path.join(pred_mask_dir, current_example[2][0])
    if os.path.exists(mask_dir):
      continue
    ##############################################
    ##          prediction: 
    ##############################################
    # create a list of (800x800) prediction distance transforms crops and softmax crops

    predDistTransform_crops=[]
    softmax_crops = []

    for idx in range(0,4):
      tmpImg =  predDistTransform[idx,:,:,:].squeeze().cpu().detach().numpy() 
      predDistTransform_crops.append(tmpImg)

    for idx in range(0,4):
      tmpImg =  softmax[idx,:,:,:].squeeze().cpu().detach().numpy() 
      softmax_crops.append(tmpImg)

    # create full-sized pred distance transform 
    mask_OrgSize = curMask_orgSize.squeeze().cpu().detach().numpy()

    height,width = mask_OrgSize.shape
    predDistTransform=np.zeros_like(mask_OrgSize)  
    predDistTransform=predDistTransform.astype(np.float32)

    tmp_predDistTransform_1=np.zeros_like(mask_OrgSize).astype(np.float32)
    tmp_predDistTransform_2=np.zeros_like(mask_OrgSize).astype(np.float32)
    tmp_predDistTransform_3=np.zeros_like(mask_OrgSize).astype(np.float32)
    tmp_predDistTransform_4=np.zeros_like(mask_OrgSize).astype(np.float32)


    tmp_predDistTransform_1[0:800, 0:800]=predDistTransform_crops[0]
    tmp_predDistTransform_2[0:800, width-800:]=predDistTransform_crops[1]
    tmp_predDistTransform_3[height-800:, 0:800]=predDistTransform_crops[2]
    tmp_predDistTransform_4[height-800:, width-800:]=predDistTransform_crops[3]

    predDistTransform = np.maximum.reduce([tmp_predDistTransform_1,tmp_predDistTransform_2,tmp_predDistTransform_3,tmp_predDistTransform_4]) 
    predDistTransform = gaussian_filter(predDistTransform, sigma=10) # gaussian blur to get rid of shadow
    pred_distanceTransform = np.copy(predDistTransform)

    # create full-sized softmax
    height,width = mask_OrgSize.shape
    softmax=np.zeros_like(mask_OrgSize)  
    softmax=softmax.astype(np.float32)

    tmp_softmax_1=np.zeros_like(mask_OrgSize).astype(np.float32)
    tmp_softmax_2=np.zeros_like(mask_OrgSize).astype(np.float32)
    tmp_softmax_3=np.zeros_like(mask_OrgSize).astype(np.float32)
    tmp_softmax_4=np.zeros_like(mask_OrgSize).astype(np.float32)

    tmp_softmax_1[0:800, 0:800]=softmax_crops[0]
    tmp_softmax_2[0:800, width-800:]=softmax_crops[1]
    tmp_softmax_3[height-800:, 0:800]=softmax_crops[2]
    tmp_softmax_4[height-800:, width-800:]=softmax_crops[3]

    tmp_softmax_1[tmp_softmax_1 == 0] = np.nan
    tmp_softmax_2[tmp_softmax_2 == 0] = np.nan
    tmp_softmax_3[tmp_softmax_3 == 0] = np.nan
    tmp_softmax_4[tmp_softmax_4 == 0] = np.nan

    # softmax = np.maximum.reduce([tmp_softmax_1,tmp_softmax_2,tmp_softmax_3,tmp_softmax_4]) 
    softmax =  np.nanmean(np.array([tmp_softmax_1,tmp_softmax_2,tmp_softmax_3,tmp_softmax_4]), axis=0)

    # find peaks, zero-out background noise
    voting4center = np.copy(pred_distanceTransform)
    voting4center[voting4center<0.001] = 0
    coord_peaks = feature.peak_local_max(voting4center, min_distance=50, exclude_border=False) #originally min_distance =5, changed to 25

    # list filenames for images in the stack
    slice_path=[]
    center_radius_list = []
    current_image_path= os.path.join(path_to_image, current_example[0][0], current_example[0][0] + '_tiles_withAnnot', current_example[1][0], current_example[2][0])
    for file in sorted(os.listdir(str(current_image_path))):
        if file.endswith('.png'):
          slice = os.path.join(str(current_image_path), file)
          slice_path.append(slice)
        if file.endswith('full_annot.plk'):
          with open(os.path.join(current_image_path,'full_annot.plk'), 'rb') as handle:
              tmpData = pickle.load(handle)
              # tmpData = json.load(f)            
              for shape in tmpData:
                  # full_annot.append(shape)
                  label_name = shape["label"]
                  # if label_name in do_not_include:
                  #     continue
                  points = shape['points']

                  # # export crop masks
                  xy = [tuple(point) for point in points]
                  (cx, cy), (px, py) = xy
                  d = math.sqrt((cx - px) ** 2 + (cy - py) ** 2)
                  center_radius_list.append((cx,cy,d,label_name))
                  
    # create detection mask using peaks and predicted radius
    centerMask = voting4center*0
    detMask = voting4center*0
    dist_transform_pred = voting4center*0
    predRadiusList = []
    detection_info = []
    detection_info2 = []
    # detMask_info = []

    size = (400,400)

    # i=0

    for i in range(coord_peaks.shape[0]):
        y, x = coord_peaks[i]
        #centerMask[y, x] = 1
        centerMask[y-10:y+10, x-10:x+10] = 1
        # predRadiusList += [voting4center[y,x] *100*800/1000 * 800/1000] # /100.0/self.resizeFactor      # why smaller?
        # predRadiusList += [voting4center[y,x] *100*1000/1000 * 1000/1000]

        left = int(x-(size[0]/2))
        left=max(left,0)
        top = int(y-(size[0]/2))
        top=max(top,0)
        right = int(x+(size[0]/2))
        right=max(right,0)
        bottom = int(y+(size[0]/2))
        bottom=max(bottom,0)

        tmpCrop = softmax[top:bottom, left:right]
        if np.isnan(np.amax(tmpCrop)):
          continue
        thresh = threshold_otsu(tmpCrop) 
        tmpCrop = tmpCrop> thresh # binarize
        tmpCrop= measure.label(tmpCrop, background=0)
        props = measure.regionprops(tmpCrop) #get the properties of the connected components

        # diameter = [prop.feret_diameter_max for prop in props]
        # diameter = [prop.equivalent_diameter for prop in props]  #diameter for connected components
        diameter = [prop.major_axis_length for prop in props]   #diameter for connected components
        if len(diameter) !=0 and max(diameter)!=0:
          radius = int(max(diameter)/2)
          predRadiusList += [radius]
          a = len(predRadiusList)-1

          tmpMask = voting4center*0
          tmpMask = create_circular_mask(tmpMask, [y, x], predRadiusList[a], value=1)

          #write bb coords to file
          # bbox = [prop.bbox for prop in props]   #bounding box coordinates for connected components

          class_name="det"
          leftBb = x-radius
          leftBb = max(leftBb,0)
          topBb = y-radius
          topBb= max(topBb,0)
          rightBb = x+radius
          rightBb = max(rightBb, 0)
          bottomBb = y+radius
          bottomBb = max(bottomBb,0)

          masked_softmax = np.ma.masked_where(tmpMask==0, softmax) 
          confidence = np.nanmean(masked_softmax)
          if confidence > 0.99:
            confidence = 0
          # bbox_info = [class_name,str(confidence) , str(leftBb), str(topBb), str(rightBb), str(bottomBb)]
          bbox_info2 = [class_name, confidence, leftBb, topBb, rightBb, bottomBb]
          # circle_info = [y,x,radius]

          # if confidence <0.05:
          #     continue
          # else:
          # detMask = create_circular_mask(detMask, [y, x], predRadiusList[a], value=a+1)
          # bbox_info = ' '.join(bbox_info)
          # detection_info.append(bbox_info)
          detection_info2.append(bbox_info2)
        # detMask_info.append(circle_info)

        # Apply non-max suppression
        NMS_bb = nms(detection_info2,conf_threshold=0.01, iou_threshold=0.3)  # set to the proper confidence threshold
        NMS_bb = NMS_bb[0]

        # print(predRadiusList)
        # print(detection_info)
        # print(detection_info2)
        # print(NMS_bb)

        # plt.imshow(masked_softmax)
        # print(tmpMask)

        ############################ attach class label ######################################################## 
        pred_bb = [leftBb, topBb, rightBb, bottomBb]
        gt_bb_list = []

        for j in range(len(center_radius_list)):
            cur_class_label_full = center_radius_list[j]
            code = str(cur_class_label_full[3])
            cy = int(center_radius_list[j][0])
            cx = int(center_radius_list[j][1])
            r = int(center_radius_list[j][2])

            gt_leftBb = cy-r
            gt_topBb = cx-r
            gt_rightBb = cy+r
            gt_bottomBb = cx+r
            gt_bb = [gt_leftBb,gt_topBb,gt_rightBb,gt_bottomBb]
            gt_bb_list.append(gt_bb)

            iou = IOU(gt_bb,pred_bb)
            a = len(gt_bb_list) - 1
            # print(iou)
            if iou>0.3:
                detMask = create_circular_mask(detMask, [cx, cy], r, value=a+1)
                dist_transform_pred = ndimage.distance_transform_edt(detMask)
                dist_transform_pred = dist_transform_pred.astype(np.uint8)


    # # fn = open(det_filename,'w')
    # fn = open(det_filename,'a')
    # for i in NMS_bb:
    #   fn.write(i + "\n")
    # fn.close()


    ### export prediction masks
    mask_dir =  os.path.join(pred_mask_dir, current_example[2][0])
    if not os.path.exists(mask_dir): 
        os.makedirs(mask_dir)
    mask_filename = os.path.join(mask_dir, 'pred_mask.png')

    if isinstance(dist_transform_pred, np.ndarray):
        dist_transform_pred_tosave = PIL.Image.fromarray((dist_transform_pred).astype(np.uint8))
    dist_transform_pred_tosave.save(mask_filename)

50/240
100/240
150/240
200/240


In [19]:
if len(NMS_bb) == 0:
  print('null')

null


In [ ]:
        if len(NMS_bb) == 0:
            dist_transform_pred=np.zeros_like(mask_OrgSize)  
            ### export prediction masks
            mask_dir =  os.path.join(pred_mask_dir, current_example[2][0])
            if not os.path.exists(mask_dir): 
                os.makedirs(mask_dir)
            mask_filename = os.path.join(mask_dir, 'pred_mask.png')

            if isinstance(dist_transform_pred, np.ndarray):
                dist_transform_pred_tosave = PIL.Image.fromarray((dist_transform_pred).astype(np.uint8))
            dist_transform_pred_tosave.save(mask_filename)

        else: